In [ ]:
!pip install kneed
from grsr_module import improved_multi_curve_feature_pruner_exp # experimental version

training_data = np.loadtxt(ucr_data_path +
                                ucr_data_name +
                                '/'+ ucr_data_name + "_TRAIN.tsv")
Y_training, X_training = training_data[:, 0].astype(int), training_data[:, 1:].astype(np.float32)
#print('X_training.shape',X_training.shape)

test_data = np.loadtxt(ucr_data_path +
                                ucr_data_name +
                                '/'+ ucr_data_name + "_TEST.tsv")
Y_test, X_test = test_data[:, 0].astype(int), test_data[:, 1:].astype(np.float32)
#print('X_test.shape',X_test.shape)

parameters = fit(X_training)
X_training_transform = transform(X_training, parameters)
X_test_transform = transform(X_test, parameters)

sclr = StandardScaler()
sclr.fit(X_training_trans)
X_training_transform_scaled = sclr.transform(X_training_transform)
X_test_transform_scaled = sclr.transform(X_test_transform)
clf = RidgeClassifierCV(np.logspace(-3,3,10))
clf.fit(X_training_transform_scaled, Y_training)
w_ridgecv = clf.coef_

# Erocket accuracy
u_tilde, first_point_non_neg_weight, knees_minus, knees_plus = improved_multi_curve_feature_pruner_exp(w_ridgecv)
X_training_transform_reduced = X_training_transform_scaled[:, u_tilde]
X_test_transform_reduced = X_test_transform_scaled[:, u_tilde]
clf_erocket = RidgeClassifierCV(np.logspace(-3,3,10))
clf_erocket.fit(X_training_transform_reduced, Y_training)
acc_erocket = clf_erocket.score(X_test_transform_reduced, Y_test)
